In [1]:
from datasets import load_dataset
import random

model_name = "cyberagent/calm3-22b-chat"

/home/setup/miniconda3/envs/vllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds=load_dataset("globis-university/aozorabunko-clean",split="train")

In [34]:
def extract_random_text(ds):
    record=ds[random.randint(0,len(ds))]
    extract_length=random.randint(100,300)
    try:
        extract_pos=random.randint(0,len(record["text"])-extract_length)
        extracted_text=record["text"][extract_pos:extract_pos+extract_length].strip()
    except:
        extracted_text=record["text"][0:100].strip()
   
    return extracted_text

In [4]:
#auto reload modules
%load_ext autoreload
%autoreload 2

from datetime import datetime
import os
import random
import json
from vllm import LLM, SamplingParams
pid = os.getpid()
seed = int(pid)+int(datetime.now().timestamp())
print("seed: ", seed)
random.seed(seed)



out_dir = "data"
os.system(f"mkdir -p {out_dir}")

current_time_no_symbols = datetime.now().strftime(
    "%Y-%m-%d %H:%M:%S").replace("-", "").replace(":", "").replace(" ", "")



2024-09-20 09:59:15,634	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


seed:  1729327369


In [5]:

max_tokens=1024
def llm_gen(model, prompt_list,
            temperature=0.7, top_k=50,
            max_tokens=max_tokens,
            ):
    outputs = model.generate(
        prompt_list,
        sampling_params=SamplingParams(
            temperature=temperature,
            max_tokens=max_tokens,
            top_k=top_k,
        )
    )
    return [i.outputs[0].text.strip() for i in outputs]


tensor_parallel_size = 1
# トークナイザーとモデルの準備
model = LLM(
    model=model_name,
    trust_remote_code=True,
    max_model_len=2000,
    tensor_parallel_size=tensor_parallel_size,
)



INFO 09-20 09:59:16 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='cyberagent/calm3-22b-chat', speculative_config=None, tokenizer='cyberagent/calm3-22b-chat', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=cyberagent/calm3-22b-chat, use_v2_block_manager=False, num_scheduler_steps=1, enable_prefix_caching=False, use_async_

Loading safetensors checkpoint shards:   0% Completed | 0/10 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  10% Completed | 1/10 [00:06<00:58,  6.48s/it]
Loading safetensors checkpoint shards:  20% Completed | 2/10 [00:26<01:56, 14.55s/it]
Loading safetensors checkpoint shards:  30% Completed | 3/10 [00:58<02:38, 22.63s/it]
Loading safetensors checkpoint shards:  40% Completed | 4/10 [01:17<02:06, 21.00s/it]
Loading safetensors checkpoint shards:  50% Completed | 5/10 [01:38<01:44, 20.99s/it]
Loading safetensors checkpoint shards:  60% Completed | 6/10 [02:10<01:39, 24.84s/it]
Loading safetensors checkpoint shards:  70% Completed | 7/10 [02:42<01:21, 27.18s/it]
Loading safetensors checkpoint shards:  80% Completed | 8/10 [03:14<00:57, 28.57s/it]
Loading safetensors checkpoint shards:  90% Completed | 9/10 [03:33<00:25, 25.81s/it]
Loading safetensors checkpoint shards: 100% Completed | 10/10 [03:55<00:00, 24.37s/it]
Loading safetensors checkpoint shards: 100% Completed | 10/10

INFO 09-20 10:03:14 model_runner.py:1008] Loading model weights took 41.9933 GB
INFO 09-20 10:03:15 gpu_executor.py:122] # GPU blocks: 1616, # CPU blocks: 227
INFO 09-20 10:03:17 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-20 10:03:17 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-20 10:03:28 model_runner.py:1430] Graph capturing finished in 11 secs.


In [6]:
import transformers
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [84]:


genres = [
    "心温まる日常会話",
    "楽しい会話",
    "ユーモラスな会話",
    "日常会話",
    "傾聴の会話",
    "相手の興味をそそる会話",
    "長めの会話",
    #"指示を聞いて答えるタイプの会話",
]


def clean_output(text):
    if text.startswith("「"):
        text = text[1:]
    if text.endswith("」"):
        text = text[:-1]
    return text


In [85]:
import copy
def ask_model(messages,model):
    messages=copy.deepcopy(messages)

    if messages[-1]["role"]=="user":
        messages.append({"role": "assistant", "content": ""})
    else:
        messages.append({"role": "user", "content": ""})
    
    prompt = tokenizer.apply_chat_template(messages, tokenize=False,)
    prompt=prompt.replace("<|im_end|>\n","")
    text=llm_gen(model, [prompt], temperature=0.7, top_k=50, max_tokens=1024)[0]
    return text

def gen_init_text(line, genre):
    system_list = [
        f"次の文章を部分的に引用しながら､{genre}のはじめのセリフを出力しなさい｡対話はタメ口で行われ､敬語は使いません｡セリフのみを出力し、それ以外は何も出力しないでください｡",
        f"次の文章を部分的に引用しながら､{genre}のはじめのセリフを出力しなさい｡対話は敬語で行われます｡セリフのみを出力し、それ以外は何も出力しないでください｡",
    ]
    system_message = random.choice(system_list)

    prompt = f"""
    #参考にする文章: {line}
    """

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt},
    ]
    text=ask_model(messages,model)
    text=clean_output(text)
    return text


In [86]:
def gen_text( genres, client,
             n_replies=8):
    genre = random.choice(genres)

    seed_text=extract_random_text(ds)
    text = gen_init_text(seed_text, genre)

    system_list = [
        f"{genre}の返答文を生成しなさい｡セリフのみを出力し、それ以外は何も出力しないでください｡",
    ]
    system_message = random.choice(system_list)
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": text}
    ]

    for j in range(n_replies):
        res = ask_model(messages, client)
        res=clean_output(res)
        if j % 2 == 1:
            messages.append({"role": "user", "content": res})
        else:
            messages.append({"role": "assistant", "content": res})

    return messages



In [87]:
#seed_text=extract_random_text(ds)
#prompt=gen_init_text(seed_text,random.choice(genres))
#print(prompt)

In [88]:

messages = gen_text(genres, model, n_replies=11)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.06it/s, est. speed input: 986.54 toks/s, output: 24.59 toks/s]


In [89]:
messages

[{'role': 'system',
  'content': '楽しい会話の返答文を生成しなさい｡セリフのみを出力し、それ以外は何も出力しないでください｡'},
 {'role': 'user',
  'content': 'なんか、ちょっと難しい話してたみたいだけど、結局、感動って大事だよね。今ってどんな芸術が一番好き？'},
 {'role': 'assistant',
  'content': '絵を描くのが好きだよ。キャンバスの中に自分の感情や物語を詰め込むことができるから。君はどう？'},
 {'role': 'user', 'content': '私も絵を描くのが好きなんだけど、最近ちょっとスランプ気味で…何かアドバイスもらえないかな？'},
 {'role': 'assistant',
  'content': 'スランプの時こそ、いつもと違う視点から物事を見てみると良いよ。自然の中を歩いたり、新しい技法を試したりするのもおすすめだよ。'},
 {'role': 'user',
  'content': 'ありがとう！自然の中を歩くの、確かに良さそうだね。今度やってみるよ。実は最近、音楽も始めたんだけど、やっぱり絵と音楽って表現の仕方が違って面白いよね。'},
 {'role': 'assistant',
  'content': 'そうだね。音楽は聴いたり演奏したりすることで瞬時に感動を伝えられるし、絵は見る人が自分のペースで感じることができる。両方とも魅力があるね。'},
 {'role': 'user',
  'content': 'うん、両方とも素晴らしいよね。でも、時々どっちも中途半端になっちゃう気がして…どっちかに絞らなきゃいけないのかな？'},
 {'role': 'assistant',
  'content': '両方やることで新しい発見があるかもしれないよ。バランスを取るのが難しいかもしれないけど、自分が楽しいと思える瞬間を見つけることが一番大事だよ。'},
 {'role': 'user', 'content': 'そうだね、楽しむことが一番だもんね。ありがとう、ちょっと元気が出たよ。'},
 {'role': 'assistant',
  'content': 'どういたしまして！またいつでも話に来てね。君の素敵な作品を見る

In [90]:

from tqdm import tqdm
import time
out_file_path = f"data/gen_mult_{current_time_no_symbols}.jsonl"

with open(out_file_path, "a") as f:
    for i in tqdm(range(10**4)):
        try:
            messages = gen_text(genres, model, n_replies=11)
        except Exception as e:
            print(e)
            time.sleep(5)
            continue
        f.write(json.dumps({"messages": messages}, ensure_ascii=False)+"\n")
        time.sleep(1)

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 7/10000 [00:48<19:04:09,  6.87s/it]


KeyboardInterrupt: 